## Ejercicio práctico

Este notebook contiene una serie de ejercicios prácticos enfocados en repasar las primeras etapas del análisis exploratorio de datos (EDA) con **PySpark**, específicamente las **estadísticas descriptivas** y la inspección básica de DataFrames.

| Comando | Función Principal | Descripción Detallada | Ejemplo Práctico |
| :--- | :--- | :--- | :--- |
| **Inspección de Estructura** | | | |
| `printSchema()` | Muestra la **estructura y tipos** de datos. | Imprime el esquema, nombres de columnas y sus tipos de datos (incluyendo nulabilidad). | \`df.printSchema()\` |
| `schema` | Obtiene el objeto **StructType** del esquema. | Devuelve el esquema del DataFrame como un objeto programable (\`StructType\`). | \`df.schema\` |
| `columns` | Lista de **nombres de columnas**. | Devuelve una lista de cadenas con los nombres de todas las columnas. | \`df.columns\` |
| `dtypes` | Lista de **nombres y tipos**. | Devuelve una lista de tuplas con el nombre de la columna y su tipo de dato. | \`df.dtypes\` |
| **Visualización y Muestreo** | | | |
| `show(n)` | Muestra las **primeras *n* filas**. | Muestra los datos del DataFrame en formato de tabla en la consola. | \`df.show(5)\` |
| `head(n)` | Devuelve las primeras *n* filas como **objetos `Row`**. | Retorna una lista de objetos \`Row\` de las primeras *n* filas para manipulación local. | \`df.head(3)\` |
| **Estadísticas Descriptivas** | | | |
| `count()` | **Cuenta** el total de filas/registros. | Devuelve el número total de filas en el DataFrame. | \`df.count()\` |
| `describe()` | Estadísticas **solo para columnas numéricas**. | Calcula \`count\`, \`mean\`, \`stddev\`, \`min\` y \`max\` para columnas numéricas. | \`df.describe().show()\` |
| `summary()` | Estadísticas para **todas las columnas**. | Proporciona estadísticas descriptivas para columnas numéricas y categóricas (incluyendo percentiles). | \`df.summary().show()\` |
| **Selección y Manipulación** | | | |
| `select()` | **Selecciona** columnas específicas. | Proyecta un nuevo DataFrame que contiene solo las columnas seleccionadas. | \`df.select('col1', 'col2').show()\` |
| `filter()` | **Filtra** filas por una condición. | Devuelve un DataFrame con las filas que cumplen la expresión booleana. | \`df.filter(df['col'] > 10).show()\` |
| `distinct()` | Elimina **filas duplicadas**. | Devuelve un DataFrame con filas únicas. | \`df.distinct().show()\` |
| `orderBy()` | **Ordena** el DataFrame. | Ordena el DataFrame según una o más columnas, de forma ascendente o descendente. | \`df.orderBy('col', ascending=False).show()\` |
| `groupBy().agg()` | **Agrupa** y **agrega** datos. | Agrupa los datos por una o más columnas y realiza operaciones de agregación (media, suma, etc.). | \`df.groupBy('col').agg({'other_col': 'mean'}).show()\` |

### Actividad 0:
Crea una sesión de Spark y lee el archivo CSV en un DataFrame de Spark


#### Cargamos el dataset

Enlace:
https://opendata-ajuntament.barcelona.cat/data/es/dataset/pad_nai_mdbas/resource/43b15838-3724-4dae-a096-1eaf4a345ef7

### Información del dataset:
https://opendata-ajuntament.barcelona.cat/data/es/dataset/pad_nai_mdbas



In [1]:
# Instalación
#pip install requests
#pip install pandas
#pip install pyspark

import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg



In [2]:
params = {'resource_id': '43b15838-3724-4dae-a096-1eaf4a345ef7', 'limit': 1067}

response = requests.get('https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search', params=params)
response.raise_for_status()

data = response.json()
records = data['result']['records']
df = pd.DataFrame.from_records(records)

df

,Codi_Districte,Nom_Districte,Nom_Barri,AEB,Seccio_Censal,Valor,Codi_Barri,_id,Any
0,1,Ciutat Vella,el Raval,1,1001,7,1,1,2024
1,1,Ciutat Vella,el Raval,1,1002,10,1,2,2024
2,1,Ciutat Vella,el Raval,2,1003,41,1,3,2024
3,1,Ciutat Vella,el Raval,2,1004,22,1,4,2024
4,1,Ciutat Vella,el Raval,3,1005,19,1,5,2024
...,...,...,...,...,...,...,...,...,...
1062,10,Sant Martí,la Verneda i la Pau,232,10139,6,73,1063,2024
1063,10,Sant Martí,la Verneda i la Pau,233,10140,9,73,1064,2024
1064,10,Sant Martí,la Verneda i la Pau,233,10141,..,73,1065,2024
1065,10,Sant Martí,la Verneda i la Pau,233,10142,12,73,1066,2024


Si queremos cargar el dataset como lo hacemos habitualmente

```python

import csv

df_2 = pd.read_csv('2024_pad_nai_mdbas.csv')


spark = (
    SparkSession.builder
    .appName("Ejercicios con PySpark Nacimiento")
    #.master("local[*]")
    .getOrCreate()
)


sdf = spark.createDataFrame(df_2)


```

Si trabajan desde Google Colab recuerden que deben cargar el dataframe, debe quedar así:



Si tienen dudas, se recomienda mirar el práctico anterior :)

In [3]:
spark = (
    SparkSession.builder
    .appName("Ejercicios con PySpark Nacimiento")
    .getOrCreate()
)

In [4]:
sdf = spark.createDataFrame(df)

### Actividad 1:
Muestra el esquema del Spark DataFrame (`sdf`) para revisar los tipos de datos inferidos.

In [5]:
sdf.printSchema()

root
 |-- Codi_Districte: string (nullable = true)
 |-- Nom_Districte: string (nullable = true)
 |-- Nom_Barri: string (nullable = true)
 |-- AEB: string (nullable = true)
 |-- Seccio_Censal: string (nullable = true)
 |-- Valor: string (nullable = true)
 |-- Codi_Barri: string (nullable = true)
 |-- _id: long (nullable = true)
 |-- Any: string (nullable = true)



### Actividad 2:
Muestra los primeros **15** registros del `sdf`.

In [6]:
sdf.show(15)

+--------------+-------------+---------+---+-------------+-----+----------+---+----+
|Codi_Districte|Nom_Districte|Nom_Barri|AEB|Seccio_Censal|Valor|Codi_Barri|_id| Any|
+--------------+-------------+---------+---+-------------+-----+----------+---+----+
|             1| Ciutat Vella| el Raval|  1|         1001|    7|         1|  1|2024|
|             1| Ciutat Vella| el Raval|  1|         1002|   10|         1|  2|2024|
|             1| Ciutat Vella| el Raval|  2|         1003|   41|         1|  3|2024|
|             1| Ciutat Vella| el Raval|  2|         1004|   22|         1|  4|2024|
|             1| Ciutat Vella| el Raval|  3|         1005|   19|         1|  5|2024|
|             1| Ciutat Vella| el Raval|  3|         1006|    6|         1|  6|2024|
|             1| Ciutat Vella| el Raval|  3|         1007|   20|         1|  7|2024|
|             1| Ciutat Vella| el Raval|  4|         1008|   46|         1|  8|2024|
|             1| Ciutat Vella| el Raval|  4|         1009|   19| 

### Actividad 3:
Muestra la lista de nombres de las columnas del DataFrame.

In [7]:
sdf.columns

['Codi_Districte',
 'Nom_Districte',
 'Nom_Barri',
 'AEB',
 'Seccio_Censal',
 'Valor',
 'Codi_Barri',
 '_id',
 'Any']

### Actividad 4:
Cambia el nombre de la columna "AEB" a "AEB_2"


Verifica si el cambio se realizó correctamente

In [8]:
sdf = sdf.withColumnRenamed('AEB', 'AEB_2')

In [9]:
sdf.columns

['Codi_Districte',
 'Nom_Districte',
 'Nom_Barri',
 'AEB_2',
 'Seccio_Censal',
 'Valor',
 'Codi_Barri',
 '_id',
 'Any']

### Actividad 5:
Muestra los tipos de datos de cada columna.

In [10]:
sdf.dtypes

[('Codi_Districte', 'string'),
 ('Nom_Districte', 'string'),
 ('Nom_Barri', 'string'),
 ('AEB_2', 'string'),
 ('Seccio_Censal', 'string'),
 ('Valor', 'string'),
 ('Codi_Barri', 'string'),
 ('_id', 'bigint'),
 ('Any', 'string')]

### Actividad 6:
Selecciona y muestra la columna `"Nom_Districte"`.

In [11]:
sdf.select("Nom_Districte").show()

+-------------+
|Nom_Districte|
+-------------+
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
| Ciutat Vella|
+-------------+
only showing top 20 rows



### Actividad 7:

Verificar la consistencia de los datos, ordenando el DataFrame por tres columnas:
1.  `Nom_Districte` (Ascendente)
2.  `Nom_Barri` (Ascendente)
3.  `Any` (Descendente)

In [12]:
sdf.orderBy(["Nom_Districte", "Nom_Barri", "Any"], ascending=[True, True, False]).show(15, False)


+--------------+-------------+-------------------------------------+-----+-------------+-----+----------+---+----+
|Codi_Districte|Nom_Districte|Nom_Barri                            |AEB_2|Seccio_Censal|Valor|Codi_Barri|_id|Any |
+--------------+-------------+-------------------------------------+-----+-------------+-----+----------+---+----+
|1             |Ciutat Vella |Sant Pere, Santa Caterina i la Ribera|17   |1049         |8    |4         |48 |2024|
|1             |Ciutat Vella |Sant Pere, Santa Caterina i la Ribera|16   |1043         |7    |4         |42 |2024|
|1             |Ciutat Vella |Sant Pere, Santa Caterina i la Ribera|16   |1044         |11   |4         |43 |2024|
|1             |Ciutat Vella |Sant Pere, Santa Caterina i la Ribera|16   |1045         |11   |4         |44 |2024|
|1             |Ciutat Vella |Sant Pere, Santa Caterina i la Ribera|16   |1046         |5    |4         |45 |2024|
|1             |Ciutat Vella |Sant Pere, Santa Caterina i la Ribera|17   |1047  

### Actividad 8:

Filtrar las filas donde en "Nom_Barri" es "Sants"

In [13]:
sdf_sants = sdf.filter(sdf["Nom_Barri"] == "Sants")

sdf_sants.show()

+--------------+--------------+---------+-----+-------------+-----+----------+---+----+
|Codi_Districte| Nom_Districte|Nom_Barri|AEB_2|Seccio_Censal|Valor|Codi_Barri|_id| Any|
+--------------+--------------+---------+-----+-------------+-----+----------+---+----+
|             3|Sants-Montjuïc|    Sants|   72|         3086|   ..|        18|316|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3087|    9|        18|317|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3088|    9|        18|318|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3089|   12|        18|319|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3090|    8|        18|320|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3091|   13|        18|321|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3092|    7|        18|322|2024|
|             3|Sants-Montjuïc|    Sants|   72|         3093|    6|        18|323|2024|
|             3|Sants-Montjuïc| 

### Actividad 9:
Calcula el número total de registros por "Nom_Barri"



In [14]:
sdf.groupBy("Nom_Barri").count().show()

+--------------------+-----+
|           Nom_Barri|count|
+--------------------+-----+
|   la Vila de Gràcia|   36|
|la Dreta de l'Eix...|   29|
|      el Barri Gòtic|    9|
|l'Antiga Esquerra...|   26|
|   la Marina de Port|   17|
|la Marina del Pra...|    1|
|              Sarrià|   16|
|         Sant Antoni|   24|
|Sant Gervasi - Ga...|   31|
|Sant Pere, Santa ...|   13|
|       Sants - Badal|   16|
|la Nova Esquerra ...|   40|
|           les Corts|   35|
|la Font de la Gua...|    7|
|Sant Gervasi - la...|   18|
|la Maternitat i S...|   15|
|       el Fort Pienc|   20|
|el Putxet i el Farró|   19|
|     les Tres Torres|   11|
|          la Bordeta|   13|
+--------------------+-----+
only showing top 20 rows



### Actividad 10:
Muestra todos los valores distintos de la columna "Nom_Barri", asegurándote de que el listado esté ordenado alfabéticamente.


In [15]:
sdf.select("Nom_Barri").distinct().orderBy("Nom_Barri").show()


+--------------------+
|           Nom_Barri|
+--------------------+
|       Baró de Viver|
|            Can Baró|
|         Can Peguera|
|           Canyelles|
|    Ciutat Meridiana|
|Diagonal Mar i el...|
|               Horta|
|         Hostafrancs|
|             Montbau|
|               Navas|
|           Pedralbes|
|               Porta|
|Provençals del Po...|
|         Sant Andreu|
|         Sant Antoni|
|Sant Genís dels A...|
|Sant Gervasi - Ga...|
|Sant Gervasi - la...|
|Sant Martí de Pro...|
|Sant Pere, Santa ...|
+--------------------+
only showing top 20 rows



### Actividad 11:

Muestra los estadisticos descriptivos de todo el sdf

In [16]:
sdf.describe().show()

+-------+-----------------+-------------------+---------------+-----------------+------------------+------------------+------------------+-----------------+------+
|summary|   Codi_Districte|      Nom_Districte|      Nom_Barri|            AEB_2|     Seccio_Censal|             Valor|        Codi_Barri|              _id|   Any|
+-------+-----------------+-------------------+---------------+-----------------+------------------+------------------+------------------+-----------------+------+
|  count|             1067|               1067|           1067|             1067|              1067|              1067|              1067|             1067|  1067|
|   mean|5.719775070290535|               NULL|           NULL|119.7535145267104|5780.3542642924085|11.102249488752555| 33.20899718837863|            534.0|2024.0|
| stddev|2.936988787662046|               NULL|           NULL|65.32565395073013|2937.5102018246203|5.2724904580880185|21.869354401513924|308.1606723772519|   0.0|
|    min|       

### Actividad 12:

Proporciona los estadisticos descriptivos de "Codi_Districte".

¿Tiene sentido lo que vemos?


In [17]:
sdf.select("Codi_Districte").describe().show()

+-------+-----------------+
|summary|   Codi_Districte|
+-------+-----------------+
|  count|             1067|
|   mean|5.719775070290535|
| stddev|2.936988787662046|
|    min|                1|
|    max|                9|
+-------+-----------------+



### Nota sobre la Actividad 12

> **Observación Crítica:** Note que, aunque PySpark calcula la media (`mean`) y la desviación estándar (`stddev`) para `Codi_Districte`, estas métricas **no son estadísticamente significativas** ni útiles.
>
> Esto se debe a que `Codi_Districte` es un **código nominal** (una etiqueta categórica). Solo el **conteo (`count`)**, el **mínimo (`min`)** y el **máximo (`max`)** tienen sentido. Para analizar la distribución de categorías, siempre es mejor usar el método **`.groupBy().count()`**.

### Actividad 13
Ingresa la última actividad que es necesaria realizar

In [18]:
spark.stop()